In [1]:
%matplotlib widget

In [2]:
from pathlib import Path
import numpy as np
import flammkuchen as fl
import pandas as pd
import tifffile as tiff

from fimpylab import LightsheetExperiment

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()
import ipywidgets as widgets

from lotr.utils import zscore
from lotr.pca import pca_and_phase, get_fictive_heading, fictive_heading_and_fit, \
        fit_phase_neurons,qap_sorting_and_phase
from circle_fit import hyper_fit
from lotr.experiment_class import LotrExperiment
import json

from lotr.plotting.color_utils import get_n_colors
from scipy.interpolate import interp1d
from scipy import signal

In [3]:
def exp_decay_kernel(tau, dt, len_rec):
    upsample = 10
    t = np.arange(len_rec * upsample) * dt / upsample
    
    decay = np.exp(-t / tau)
    decay /= np.sum(decay)
    return decay

In [4]:
path = Path(r"\\Funes\Shared\experiments\E0071_lotr\full_ring\210314_f1\210314_f1_natmov")
master =  Path(r"\\Funes\Shared\experiments\E0071_lotr\full_ring")
files = list(master.glob("*/*_f*"))
path = files[0] 
print(path)

\\Funes\Shared\experiments\E0071_lotr\full_ring\210314_f0\210314_f0_natmov


In [5]:
traces = fl.load(path / "filtered_traces.h5", "/detr").T
try:
    selected = fl.load(path / "selected.h5")
except:
    print("no selected")

reg_df = fl.load(path / "motor_regressors.h5")
cc_motor = reg_df["all_bias_abs"].values
cc_motor_integr = reg_df["all_bias_abs_dfdt"].values
try:
    coords = fl.load(path / "data_from_suite2p_unfiltered.h5", "/coords")
    anat = fl.load(path / "data_from_suite2p_unfiltered.h5", "/anatomy_stack")
except:
    coords = fl.load(path / "data_from_suite2p_cells.h5", "/coords")
    anat = fl.load(path / "data_from_suite2p_cells.h5", "/anatomy_stack")

df = fl.load(path / "bouts_df.h5")# exp.get_bout_properties()
exp = LotrExperiment(path)
fn = int(exp.fn)
beh_df = exp.behavior_log

t_start_s = 150
t_lims = [150, -50]
t_slice = slice(*t_lims)

In [6]:
np.shape(traces)

(471, 9900)

In [7]:
stim_log = exp.stimulus_log
beh_log = exp.behavior_log
stim_log

,bg_x,bg_y,bg_theta,bg_current_phase,t
0,227.266079,-102.809200,2.140722,0,80.012243
1,227.280363,-102.827771,2.146492,6,80.034231
2,227.290931,-102.833984,2.144406,9,80.051221
3,227.294517,-102.840270,2.148816,11,80.064214
4,227.311337,-102.857349,2.148339,14,80.082203
...,...,...,...,...,...
112815,227.755157,-119.819555,28.250523,53901,1979.922760
112816,227.755839,-119.833585,28.248184,53904,1979.936752
112817,227.754705,-119.842943,28.250142,53907,1979.953050
112818,227.753325,-119.847854,28.251980,53909,1979.964172


In [532]:
try:
    theta = np.asarray(stim_log['bg_theta'])
except:
    theta1 = np.asarray(stim_log['open_loop_theta'])
    theta1[np.where(np.isnan(theta1))[0]] = 0
    theta2 = np.asarray(stim_log['closed_loop_theta'])
    theta2[np.where(np.isnan(theta2))[0]] = 0
    theta = theta1 + theta2
#theta = np.asarray(stim_log['cl2D_vel'])
stim_t = np.asarray(stim_log.t)

theta_wrapped = np.mod(theta, np.pi * 2) - np.pi
theta_diff = np.zeros_like(theta)
theta_diff[1:] = np.diff(theta)
theta_diff[theta_diff < -0.5] = 0
theta_diff *= 50

pos_theta = np.copy(theta_diff)
pos_theta[pos_theta < 0] = 0

neg_theta = np.copy(theta_diff)
neg_theta[neg_theta > 0] = 0

theta_off = np.zeros_like(theta)
theta_off[1:] = np.diff(theta)

In [533]:
fig, ax = plt.subplots(1,1)
ax.plot(stim_t, (theta / 5)+ 5)
ax.plot(stim_t, theta_wrapped)
ax.plot(stim_t, (theta_diff)-10)
ax.plot(beh_log.t, beh_log.tail_sum - 30)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [534]:
fs = exp.fn
print(fs)
dt_imaging = 1 / fs
int_fact = 200
t_imaging = np.arange(traces.shape[1])/fs
num_traces, len_rec = np.shape(traces)

ca_kernel = exp_decay_kernel(tau=0.8, dt=1/fs, len_rec=len_rec)
ca_kernel = ca_kernel[0:500]

5


In [535]:
# generating regressors
from lotr.default_vals import REGRESSOR_TAU_S, TURN_BIAS

tau_fs = REGRESSOR_TAU_S * fn
kernel = np.exp(-np.arange(1000) / tau_fs)
t_imaging_int = np.arange(traces.shape[1]*int_fact)*dt_imaging/int_fact

theta_int = interp1d(stim_log.t, theta, bounds_error=False, fill_value=0)(t_imaging_int)
theta_reg = signal.decimate(theta_int, int_fact, ftype="fir")
theta_reg_conv = np.convolve(theta_reg, ca_kernel, mode='full') * 2
kernel_size = np.shape(ca_kernel)[0] - 1
theta_reg_conv = theta_reg_conv[0:-kernel_size]

theta_wrp_int = interp1d(stim_log.t, theta_wrapped, bounds_error=False, fill_value=0)(t_imaging_int)
theta_wrp_reg = signal.decimate(theta_wrp_int, int_fact, ftype="fir")
theta_wrp_conv = np.convolve(theta_wrp_reg, ca_kernel, mode='full') * 2
theta_wrp_conv = theta_wrp_conv[0:-kernel_size]

theta_diff_int = interp1d(stim_log.t, theta_diff, bounds_error=False, fill_value=0)(t_imaging_int)
theta_diff_reg = signal.decimate(theta_diff_int, int_fact, ftype="fir")
theta_diff_conv = np.convolve(theta_diff_reg, ca_kernel, mode='full') * 2
theta_diff_conv = theta_diff_conv[0:-kernel_size]

theta_off_int = interp1d(stim_log.t, theta_off, bounds_error=False, fill_value=0)(t_imaging_int)
theta_off_reg = signal.decimate(theta_off_int, int_fact, ftype="fir")
theta_off_conv = np.convolve(theta_off_reg, ca_kernel, mode='full') * 2
theta_off_conv = theta_off_conv[0:-kernel_size]

In [536]:
fig, ax = plt.subplots(2,1)
stim_t_conv = np.arange(np.shape(theta_reg_conv)[0]) / 5
ax[0].plot(stim_t_conv, theta_reg_conv / 5 + 5)
ax[0].plot(stim_t_conv, theta_wrp_conv)
ax[0].plot(stim_t_conv, theta_diff_conv - 5)
ax[0].plot(beh_log.t, beh_log.tail_sum - 15)
ax[0].plot(stim_t_conv, traces[15] - 30)
ax[0].plot(stim_t_conv, theta_off_conv/5 - 25)
ax[0].set_xlim(0, stim_t_conv[-1])

ax[1].imshow(traces, vmin=-1, vmax=2, extent=[0, 1500, 0, 500])
ax[0].axis('off')
ax[1].axis('off')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 1500.0, 0.0, 500.0)

In [537]:
theta_traces = np.dot(traces, theta_reg_conv) - num_traces * np.mean(traces, 1) * np.mean(theta_reg_conv)
theta_traces /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(theta_reg_conv)

theta_wrp_traces = np.dot(traces, theta_wrp_conv) - num_traces * np.mean(traces, 1) * np.mean(theta_wrp_conv)
theta_wrp_traces /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(theta_wrp_conv)

theta_diff_traces = np.dot(traces, theta_diff_conv) - num_traces * np.mean(traces, 1) * np.mean(theta_diff_conv)
theta_diff_traces /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(theta_diff_conv)

theta_off_traces = np.dot(traces, theta_off_conv) - num_traces * np.mean(traces, 1) * np.mean(theta_diff_conv)
theta_off_traces /= (traces.shape[1] - 1) * np.std(traces, 1) * np.std(theta_off_conv)

In [538]:
reg_values = {
    'theta_reg': theta_traces,
    'theta_wrp_reg': theta_wrp_traces,
    'theta_diff_reg': theta_diff_traces,
    'theta_off_reg': theta_off_traces,
    }
fl.save(str(path / 'stimulus_regression_values.h5'), reg_values)

In [539]:
fig_anatomy2, ax_anatomy2 = plt.subplots(1, 4, figsize=(9, 3))

ax_anatomy2[0].scatter(coords[:, 1], coords[:, 2], c=theta_traces, cmap='coolwarm', s=3, vmin=-1, vmax=1)
ax_anatomy2[0].axis('off')

ax_anatomy2[1].scatter(coords[:, 1], coords[:, 2], c=theta_wrp_traces, cmap='coolwarm', s=3, vmin=-1, vmax=1)
ax_anatomy2[1].axis('off')

ax_anatomy2[2].scatter(coords[:, 1], coords[:, 2], c=theta_diff_traces, cmap='coolwarm', s=3, vmin=-1, vmax=1)
ax_anatomy2[2].axis('off')

ax_anatomy2[3].scatter(coords[:, 1], coords[:, 2], c=theta_off_traces, cmap='coolwarm', s=3, vmin=-1, vmax=1)
ax_anatomy2[3].axis('off')

file_name = 'theta_regressors_rois.jpg'
fig_anatomy2.savefig(path / file_name)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …